In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
WORKDIR = "csv/"
EXT = '.csv'
tickers = ['IUSA.MI']

In [4]:
ticker_file = WORKDIR + tickers[0] + EXT

#df = pd.read_csv(ticker_file, index_col=1, parse_dates=True)
df = pd.read_csv(ticker_file, usecols=[1,2,3,4,5,6], index_col=0, parse_dates=True)

In [5]:
df.head()

,open,high,low,close,volume
dateTime,,,,,
2004-11-05,9.258,9.30,9.19,9.290,60589.0
2004-12-05,9.175,9.38,9.12,9.140,124040.0
2004-05-13,9.265,9.29,9.23,9.290,123133.0
2004-05-14,9.263,9.31,9.19,9.234,28694.0
2004-05-17,9.033,9.15,8.99,9.031,537522.0


In [51]:
df2 = pd.read_excel('infoproviders.xlsx').iloc[6:,:]
df2.columns = df2.iloc[0,:].tolist()
df2 = df2.iloc[1:,:]
df2.set_index('N.',inplace=True)


In [52]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 968 entries, 283 to 782
Data columns (total 11 columns):
ISIN                       968 non-null object
Nome                       968 non-null object
Local Market TIDM          968 non-null object
Reuters RIC (Italy)        968 non-null object
Bloomerg Ticker (Italy)    968 non-null object
Indice Benchmark           937 non-null object
TER                        968 non-null object
Area Benchmark             968 non-null object
Strumento                  968 non-null object
Emittente                  968 non-null object
Tick                       968 non-null object
dtypes: object(11)
memory usage: 90.8+ KB
